In [1]:
import pandas as pd
import numpy as np
import re

Load data:

In [2]:
data_dir = '//imsbnas.ethz.ch/Sauer1/users/Xavier/3_Karl_data/'
metabolites = pd.read_excel(data_dir+"merged_metabolites.xlsx",index_col="name")
reactions = open(data_dir+'reactions_GS2.txt').read().splitlines()
mapping = pd.read_table(data_dir+'bigg_models_reactions.txt',index_col='bigg_id')
regs = pd.read_excel(data_dir+'SMRN_allorgs.xlsx',index_col='EC_number')

Calculate the metabolite variance (metabolites with higher variances are suposed to endorse more allosteric interactions).

In [3]:
variances = np.nanvar(metabolites,1)

For each reaction, look at metabolites that have been previously reported in any organism. Incorporate this prior data in a logistic regression to calculate the probability of each metabolite to be a real regulator.

In [4]:
threshold = 100 #100 candidates out of 284*2
shortlist = pd.DataFrame(columns=['metab','mode','scores','rxn_id'])
for rxn in reactions:
    rxn_df = pd.DataFrame({'metab':list(metabolites.index)+list(metabolites.index),'mode':['+']*metabolites.shape[0]+['-']*metabolites.shape[0],\
              'scores':list(variances)+list(variances),'rxn_id':[rxn.lower()]*metabolites.shape[0]*2,\
              'variance':list(variances)+list(variances),'n_alloreg':np.zeros(metabolites.shape[0]*2)})
    ec = re.findall('[1-6]+.[0-9]+.[0-9]+.[0-9]+',mapping.loc[rxn,'database_links'])
    rxn_regs = pd.DataFrame(columns=regs.columns)
    for i in ec:
        if i in regs.index:
            rxn_regs = rxn_regs.append(regs.loc[ec],ignore_index=True)
    n_allorgs = [np.sum(np.logical_and([rxn_regs['metab']==rxn_df.loc[s,'metab']],[rxn_regs['Mode']==rxn_df.loc[s,'mode']])) for s in list(rxn_df.index.values)]
    rxn_df['n_alloreg'] = n_allorgs
    n_allorgs = np.array(n_allorgs)*5.0+1.0
    rxn_df['scores'] = rxn_df['scores']*n_allorgs
    rxn_df = rxn_df.sort_values(by='scores',ascending=False)
    shortlist = shortlist.append(rxn_df.iloc[0:threshold,:],ignore_index=True)
shortlist

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if __name__ == '__main__':


,metab,mode,n_alloreg,rxn_id,scores,variance
0,akg,-,0.0,a5piso,24.024672,24.024672
1,akg,+,0.0,a5piso,24.024672,24.024672
2,2obut,+,0.0,a5piso,18.825048,18.825048
3,2obut,-,0.0,a5piso,18.825048,18.825048
4,mev__R,+,0.0,a5piso,15.135206,15.135206
5,mev__R,-,0.0,a5piso,15.135206,15.135206
6,5aizc,-,0.0,a5piso,6.474808,6.474808
7,5aizc,+,0.0,a5piso,6.474808,6.474808
8,3php,+,0.0,a5piso,5.770734,5.770734
9,3php,-,0.0,a5piso,5.770734,5.770734


Save shorlist

In [5]:
shortlist.to_csv('//imsbnas.ethz.ch/Sauer1/users/Xavier/13_Karl_pairwise_1week/shortlist.csv')